In [1]:
import featuretools as ft
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

C:\Users\hp\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df=pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
#df=df.drop(['EmployeeNumber','EmployeeCount','StandardHours','Over18'],axis=1)
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [4]:
#Binning
df['MonthlyIncome']=pd.cut(df['MonthlyIncome'],bins=4,labels=['Very low','Low','Moderate','High'])
#Binning
df['Age']=pd.cut(df['Age'],bins=3,labels=['Youth','Middle Aged','Elderly'])

In [5]:
df_num=df.select_dtypes(include=['int64','float'])
df_cat=df.select_dtypes(exclude=['int64','float'])
df_cat.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'EducationField',
       'Gender', 'JobRole', 'MaritalStatus', 'MonthlyIncome', 'Over18',
       'OverTime'],
      dtype='object')

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df_cat['MonthlyIncome']=le.fit_transform(df['MonthlyIncome'])
df_cat['Age']=le.fit_transform(df['Age'])
df_cat['BusinessTravel']=le.fit_transform(df['BusinessTravel'])
df_cat['Department']=le.fit_transform(df['Department'])
df_cat['EducationField']=le.fit_transform(df['EducationField'])
df_cat['Gender']=le.fit_transform(df['Gender'])
df_cat['JobRole']=le.fit_transform(df['JobRole'])
df_cat['MaritalStatus']=le.fit_transform(df['MaritalStatus'])
df_cat['OverTime']=le.fit_transform(df['OverTime'])

In [7]:
df1=pd.concat([df_num,df_cat],axis=1)
df1=df1.drop(['EmployeeCount','StandardHours','Over18'],axis=1)


In [8]:
df1['Attrition']=df1['Attrition'].replace({'Yes':1,'No':0})


In [9]:

#Scale the input
from sklearn.preprocessing import StandardScaler
x=df1.drop('Attrition',axis=1)
y=df1['Attrition']
x_scaler=StandardScaler()
x_std=x_scaler.fit_transform(x)


In [10]:
##Feature tools

In [11]:
es = ft.EntitySet(id="Employees")

# Add the entire data table as an entity
es.entity_from_dataframe("Employee",
                         dataframe=x,
                         index="Employee_index")

es['Employee']

Entity: Employee
  Variables:
    Employee_index (dtype: index)
    DailyRate (dtype: numeric)
    DistanceFromHome (dtype: numeric)
    Education (dtype: numeric)
    EmployeeNumber (dtype: numeric)
    EnvironmentSatisfaction (dtype: numeric)
    HourlyRate (dtype: numeric)
    JobInvolvement (dtype: numeric)
    JobLevel (dtype: numeric)
    JobSatisfaction (dtype: numeric)
    MonthlyRate (dtype: numeric)
    NumCompaniesWorked (dtype: numeric)
    PercentSalaryHike (dtype: numeric)
    PerformanceRating (dtype: numeric)
    RelationshipSatisfaction (dtype: numeric)
    StockOptionLevel (dtype: numeric)
    TotalWorkingYears (dtype: numeric)
    TrainingTimesLastYear (dtype: numeric)
    WorkLifeBalance (dtype: numeric)
    YearsAtCompany (dtype: numeric)
    YearsInCurrentRole (dtype: numeric)
    YearsSinceLastPromotion (dtype: numeric)
    YearsWithCurrManager (dtype: numeric)
    Age (dtype: numeric)
    BusinessTravel (dtype: numeric)
    Department (dtype: numeric)
    Educat

In [12]:
# create a new "products" entity
es.normalize_entity(new_entity_id="Attritions",
                    base_entity_id="Employee",
                    index="EmployeeNumber",additional_variables=['MonthlyIncome','OverTime','TotalWorkingYears','DailyRate','HourlyRate','Age','DistanceFromHome'])

es['Attritions'].df.head()

,EmployeeNumber,MonthlyIncome,OverTime,TotalWorkingYears,DailyRate,HourlyRate,Age,DistanceFromHome
1,1,1,1,8,1102,94,1,1
2,2,3,0,10,279,61,0,8
4,4,3,1,7,1373,92,1,2
5,5,3,1,8,1392,56,1,3
7,7,3,0,6,591,40,2,2


In [13]:
feature_matrix, feature_names = ft.dfs(entityset=es, target_entity='Attritions', verbose = 2,n_jobs = -1,
                                       max_depth = 2)

feature_matrix.head()

Built 146 features
EntitySet scattered to 4 workers in 7 seconds
Elapsed: 00:04 | Progress: 100%|██████████


,MonthlyIncome,OverTime,TotalWorkingYears,DailyRate,HourlyRate,Age,DistanceFromHome,COUNT(Employee),MAX(Employee.BusinessTravel),MAX(Employee.Department),...,SUM(Employee.PercentSalaryHike),SUM(Employee.PerformanceRating),SUM(Employee.RelationshipSatisfaction),SUM(Employee.StockOptionLevel),SUM(Employee.TrainingTimesLastYear),SUM(Employee.WorkLifeBalance),SUM(Employee.YearsAtCompany),SUM(Employee.YearsInCurrentRole),SUM(Employee.YearsSinceLastPromotion),SUM(Employee.YearsWithCurrManager)
EmployeeNumber,,,,,,,,,,,,,,,,,,,,,
1,1,1,8,1102,94,1,1,1,2,2,...,11,3,1,0,0,1,6,4,0,5
2,3,0,10,279,61,0,8,1,1,1,...,23,4,4,1,3,3,10,7,1,7
4,3,1,7,1373,92,1,2,1,2,1,...,15,3,2,0,3,3,0,0,0,0
5,3,1,8,1392,56,1,3,1,1,1,...,11,3,3,0,3,3,8,7,3,0
7,3,0,6,591,40,2,2,1,2,1,...,12,3,4,1,3,3,2,2,2,2


In [14]:
feature_matrix = pd.get_dummies(feature_matrix).reset_index()
feature_matrix.head()

,EmployeeNumber,MonthlyIncome,OverTime,TotalWorkingYears,DailyRate,HourlyRate,Age,DistanceFromHome,COUNT(Employee),MAX(Employee.BusinessTravel),...,SUM(Employee.PercentSalaryHike),SUM(Employee.PerformanceRating),SUM(Employee.RelationshipSatisfaction),SUM(Employee.StockOptionLevel),SUM(Employee.TrainingTimesLastYear),SUM(Employee.WorkLifeBalance),SUM(Employee.YearsAtCompany),SUM(Employee.YearsInCurrentRole),SUM(Employee.YearsSinceLastPromotion),SUM(Employee.YearsWithCurrManager)
0,1,1,1,8,1102,94,1,1,1,2,...,11,3,1,0,0,1,6,4,0,5
1,2,3,0,10,279,61,0,8,1,1,...,23,4,4,1,3,3,10,7,1,7
2,4,3,1,7,1373,92,1,2,1,2,...,15,3,2,0,3,3,0,0,0,0
3,5,3,1,8,1392,56,1,3,1,1,...,11,3,3,0,3,3,8,7,3,0
4,7,3,0,6,591,40,2,2,1,2,...,12,3,4,1,3,3,2,2,2,2


In [15]:
feature_matrix.columns

Index(['EmployeeNumber', 'MonthlyIncome', 'OverTime', 'TotalWorkingYears',
       'DailyRate', 'HourlyRate', 'Age', 'DistanceFromHome', 'COUNT(Employee)',
       'MAX(Employee.BusinessTravel)',
       ...
       'SUM(Employee.PercentSalaryHike)', 'SUM(Employee.PerformanceRating)',
       'SUM(Employee.RelationshipSatisfaction)',
       'SUM(Employee.StockOptionLevel)', 'SUM(Employee.TrainingTimesLastYear)',
       'SUM(Employee.WorkLifeBalance)', 'SUM(Employee.YearsAtCompany)',
       'SUM(Employee.YearsInCurrentRole)',
       'SUM(Employee.YearsSinceLastPromotion)',
       'SUM(Employee.YearsWithCurrManager)'],
      dtype='object', length=147)

In [16]:
feature_matrix['SUM(Employee.YearsWithCurrManager)'].values

array([5, 7, 0, ..., 3, 8, 2], dtype=int64)

In [17]:
df['YearsWithCurrManager'].values

array([5, 7, 0, ..., 3, 8, 2], dtype=int64)

In [18]:
feature_matrix=feature_matrix.dropna(axis=1)

In [19]:
feature_matrix.shape

(1470, 101)

In [20]:

#Scale the input
from sklearn.preprocessing import StandardScaler
x_scaler=StandardScaler()
x_std1=x_scaler.fit_transform(feature_matrix)


In [21]:
feature_matrix

,EmployeeNumber,MonthlyIncome,OverTime,TotalWorkingYears,DailyRate,HourlyRate,Age,DistanceFromHome,COUNT(Employee),MAX(Employee.BusinessTravel),...,SUM(Employee.PercentSalaryHike),SUM(Employee.PerformanceRating),SUM(Employee.RelationshipSatisfaction),SUM(Employee.StockOptionLevel),SUM(Employee.TrainingTimesLastYear),SUM(Employee.WorkLifeBalance),SUM(Employee.YearsAtCompany),SUM(Employee.YearsInCurrentRole),SUM(Employee.YearsSinceLastPromotion),SUM(Employee.YearsWithCurrManager)
0,1,1,1,8,1102,94,1,1,1,2,...,11,3,1,0,0,1,6,4,0,5
1,2,3,0,10,279,61,0,8,1,1,...,23,4,4,1,3,3,10,7,1,7
2,4,3,1,7,1373,92,1,2,1,2,...,15,3,2,0,3,3,0,0,0,0
3,5,3,1,8,1392,56,1,3,1,1,...,11,3,3,0,3,3,8,7,3,0
4,7,3,0,6,591,40,2,2,1,2,...,12,3,4,1,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,2061,3,0,17,884,41,1,23,1,1,...,17,3,3,1,3,3,5,2,0,3
1466,2062,1,0,9,613,42,1,6,1,2,...,15,3,1,1,5,3,7,7,1,7
1467,2064,1,1,6,155,87,2,4,1,2,...,20,4,2,1,0,3,6,2,0,3
1468,2065,3,0,17,1023,63,0,2,1,1,...,14,3,4,0,3,2,9,6,0,8


In [22]:
#p-values using SelectKBest
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
skb = SelectKBest(f_classif,k=6)
skb.fit_transform(feature_matrix,y)
skb.pvalues_



df1=pd.DataFrame()
df1['Skb']=skb.pvalues_
df1['col']=feature_matrix.columns
df1[df1['Skb']<0.05].sort_values(by='Skb',ascending=False).head(50)['col'].values

array(['DailyRate', 'MIN(Employee.TrainingTimesLastYear)',
       'MEAN(Employee.TrainingTimesLastYear)',
       'SUM(Employee.TrainingTimesLastYear)',
       'MAX(Employee.TrainingTimesLastYear)',
       'MIN(Employee.WorkLifeBalance)', 'SUM(Employee.WorkLifeBalance)',
       'MEAN(Employee.WorkLifeBalance)', 'MAX(Employee.WorkLifeBalance)',
       'MEAN(Employee.Department)', 'SUM(Employee.Department)',
       'MIN(Employee.Department)', 'MAX(Employee.Department)',
       'MAX(Employee.JobRole)', 'MIN(Employee.JobRole)',
       'MEAN(Employee.JobRole)', 'SUM(Employee.JobRole)',
       'DistanceFromHome', 'MIN(Employee.EnvironmentSatisfaction)',
       'SUM(Employee.EnvironmentSatisfaction)',
       'MAX(Employee.EnvironmentSatisfaction)',
       'MEAN(Employee.EnvironmentSatisfaction)',
       'SUM(Employee.JobSatisfaction)', 'MAX(Employee.JobSatisfaction)',
       'MIN(Employee.JobSatisfaction)', 'MEAN(Employee.JobSatisfaction)',
       'MonthlyIncome', 'MEAN(Employee.JobInvolvement

In [23]:
new=feature_matrix[['DailyRate', 'MIN(Employee.TrainingTimesLastYear)',
       'MEAN(Employee.TrainingTimesLastYear)',
       'SUM(Employee.TrainingTimesLastYear)',
       'MAX(Employee.TrainingTimesLastYear)',
       'MIN(Employee.WorkLifeBalance)', 'SUM(Employee.WorkLifeBalance)',
       'MEAN(Employee.WorkLifeBalance)', 'MAX(Employee.WorkLifeBalance)',
       'MEAN(Employee.Department)', 'SUM(Employee.Department)',
       'MIN(Employee.Department)', 'MAX(Employee.Department)',
       'MAX(Employee.JobRole)', 'MIN(Employee.JobRole)',
       'MEAN(Employee.JobRole)', 'SUM(Employee.JobRole)',
       'DistanceFromHome', 'MIN(Employee.EnvironmentSatisfaction)',
       'SUM(Employee.EnvironmentSatisfaction)',
       'MAX(Employee.EnvironmentSatisfaction)',
       'MEAN(Employee.EnvironmentSatisfaction)',
       'SUM(Employee.JobSatisfaction)', 'MAX(Employee.JobSatisfaction)',
       'MIN(Employee.JobSatisfaction)', 'MEAN(Employee.JobSatisfaction)',
       'MonthlyIncome', 'MEAN(Employee.JobInvolvement)',
       'MAX(Employee.JobInvolvement)', 'MIN(Employee.JobInvolvement)',
       'SUM(Employee.JobInvolvement)', 'MIN(Employee.YearsAtCompany)',
       'SUM(Employee.YearsAtCompany)', 'MEAN(Employee.YearsAtCompany)',
       'MAX(Employee.YearsAtCompany)', 'SUM(Employee.StockOptionLevel)',
       'MAX(Employee.StockOptionLevel)', 'MIN(Employee.StockOptionLevel)',
       'MEAN(Employee.StockOptionLevel)', 'Age',
       'MEAN(Employee.YearsWithCurrManager)',
       'MIN(Employee.YearsWithCurrManager)',
       'MAX(Employee.YearsWithCurrManager)',
       'SUM(Employee.YearsWithCurrManager)',
       'MIN(Employee.YearsInCurrentRole)',
       'MEAN(Employee.YearsInCurrentRole)',
       'SUM(Employee.YearsInCurrentRole)',
       'MAX(Employee.YearsInCurrentRole)', 'MEAN(Employee.MaritalStatus)',
       'MIN(Employee.MaritalStatus)']]

In [24]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import KFold,cross_val_score

from sklearn.metrics import confusion_matrix,classification_report, roc_auc_score,roc_curve,accuracy_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(new,y,test_size=0.30,random_state=3)
lr=LogisticRegression()

#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
ytest_prob = lr.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))






cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "LR FT SELECT"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

LR_FT_SELECT =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

LR_FT_SELECT

Confusion: [[362   4]
 [ 66   9]]

Accuracy Train:  0.8551992225461613
Accuracy Test: 0.8412698412698413
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.99      0.91       366
           1       0.69      0.12      0.20        75

    accuracy                           0.84       441
   macro avg       0.77      0.55      0.56       441
weighted avg       0.82      0.84      0.79       441

AUC Test:  0.7212386156648452
True positives: 9
True negatives: 362
False positives (Type I error): 4
False negatives (Type II error): 66


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,LR FT SELECT,70,4,66,0.69,0.12,0.86,0.84,0.2,0.72


In [25]:

X_train,X_test,y_train,y_test=train_test_split(new,y,test_size=0.30,random_state=3,stratify=y)
lr=LogisticRegression()
lr_Bag=BaggingClassifier(base_estimator=lr,n_estimators=91,random_state=0)
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr_Bag.fit(X_train,y_train)
y_pred = lr_Bag.predict(X_test)
ytest_prob = lr_Bag.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr_Bag.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))



cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "Bag_LR_SELECT"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr_Bag.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

bag_LR_SELECT =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

bag_LR_SELECT


Confusion: [[369   1]
 [ 61  10]]

Accuracy Train:  0.8551992225461613
Accuracy Test: 0.8594104308390023
Classification Report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92       370
           1       0.91      0.14      0.24        71

    accuracy                           0.86       441
   macro avg       0.88      0.57      0.58       441
weighted avg       0.87      0.86      0.81       441

AUC Test:  0.6950513894175866
True positives: 10
True negatives: 369
False positives (Type I error): 1
False negatives (Type II error): 61


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,Bag_LR_SELECT,62,1,61,0.91,0.14,0.86,0.86,0.24,0.7


In [26]:
lr=LogisticRegression()
lr_Ada=AdaBoostClassifier(base_estimator=lr,n_estimators=7,random_state=0)
X_train,X_test,y_train,y_test=train_test_split(new,y,test_size=0.30,random_state=3)
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr_Ada.fit(X_train,y_train)
y_pred = lr_Ada.predict(X_test)
ytest_prob = lr_Ada.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr_Ada.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))




cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "Ada_LR_SELECT"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr_Ada.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

Ada_LR_SELECT =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

Ada_LR_SELECT


Confusion: [[364   2]
 [ 62  13]]

Accuracy Train:  0.8542274052478134
Accuracy Test: 0.854875283446712
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.99      0.92       366
           1       0.87      0.17      0.29        75

    accuracy                           0.85       441
   macro avg       0.86      0.58      0.60       441
weighted avg       0.86      0.85      0.81       441

AUC Test:  0.717959927140255
True positives: 13
True negatives: 364
False positives (Type I error): 2
False negatives (Type II error): 62


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,Ada_LR_SELECT,64,2,62,0.87,0.17,0.85,0.85,0.29,0.72


In [27]:
import statsmodels.api as sm
#Backward Elimination
cols = list(feature_matrix.columns)
pmax = 1
while (len(cols)>0):
    p=[]
    X_1 = feature_matrix[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p=pd.Series(model.pvalues.values[:],index=cols)
    pmax=max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

['OverTime', 'Age', 'DistanceFromHome', 'COUNT(Employee)', 'MAX(Employee.Department)', 'MAX(Employee.EnvironmentSatisfaction)', 'MAX(Employee.Gender)', 'MAX(Employee.JobInvolvement)', 'MAX(Employee.JobLevel)', 'MAX(Employee.JobSatisfaction)', 'MAX(Employee.MaritalStatus)', 'MAX(Employee.NumCompaniesWorked)', 'MAX(Employee.RelationshipSatisfaction)', 'MAX(Employee.StockOptionLevel)', 'MAX(Employee.WorkLifeBalance)', 'MAX(Employee.YearsInCurrentRole)', 'MAX(Employee.YearsSinceLastPromotion)', 'MAX(Employee.YearsWithCurrManager)', 'MEAN(Employee.Department)', 'MEAN(Employee.EnvironmentSatisfaction)', 'MEAN(Employee.Gender)', 'MEAN(Employee.JobInvolvement)', 'MEAN(Employee.JobLevel)', 'MEAN(Employee.JobSatisfaction)', 'MEAN(Employee.MaritalStatus)', 'MEAN(Employee.NumCompaniesWorked)', 'MEAN(Employee.RelationshipSatisfaction)', 'MEAN(Employee.StockOptionLevel)', 'MEAN(Employee.WorkLifeBalance)', 'MEAN(Employee.YearsInCurrentRole)', 'MEAN(Employee.YearsSinceLastPromotion)', 'MEAN(Employee.Y

In [28]:
new1=feature_matrix[['OverTime', 'Age', 'DistanceFromHome', 'COUNT(Employee)', 'MAX(Employee.Department)', 'MAX(Employee.EnvironmentSatisfaction)', 'MAX(Employee.Gender)', 'MAX(Employee.JobInvolvement)', 'MAX(Employee.JobLevel)', 'MAX(Employee.JobSatisfaction)', 'MAX(Employee.MaritalStatus)', 'MAX(Employee.NumCompaniesWorked)', 'MAX(Employee.RelationshipSatisfaction)', 'MAX(Employee.StockOptionLevel)', 'MAX(Employee.WorkLifeBalance)', 'MAX(Employee.YearsInCurrentRole)', 'MAX(Employee.YearsSinceLastPromotion)', 'MAX(Employee.YearsWithCurrManager)', 'MEAN(Employee.Department)', 'MEAN(Employee.EnvironmentSatisfaction)', 'MEAN(Employee.Gender)', 'MEAN(Employee.JobInvolvement)', 'MEAN(Employee.JobLevel)', 'MEAN(Employee.JobSatisfaction)', 'MEAN(Employee.MaritalStatus)', 'MEAN(Employee.NumCompaniesWorked)', 'MEAN(Employee.RelationshipSatisfaction)', 'MEAN(Employee.StockOptionLevel)', 'MEAN(Employee.WorkLifeBalance)', 'MEAN(Employee.YearsInCurrentRole)', 'MEAN(Employee.YearsSinceLastPromotion)', 'MEAN(Employee.YearsWithCurrManager)', 'MIN(Employee.Department)', 'MIN(Employee.EnvironmentSatisfaction)', 'MIN(Employee.Gender)', 'MIN(Employee.JobInvolvement)', 'MIN(Employee.JobLevel)', 'MIN(Employee.JobSatisfaction)', 'MIN(Employee.MaritalStatus)', 'MIN(Employee.NumCompaniesWorked)', 'MIN(Employee.RelationshipSatisfaction)', 'MIN(Employee.StockOptionLevel)', 'MIN(Employee.WorkLifeBalance)', 'MIN(Employee.YearsInCurrentRole)', 'MIN(Employee.YearsSinceLastPromotion)', 'MIN(Employee.YearsWithCurrManager)', 'SUM(Employee.Department)', 'SUM(Employee.EnvironmentSatisfaction)', 'SUM(Employee.Gender)', 'SUM(Employee.JobInvolvement)', 'SUM(Employee.JobLevel)', 'SUM(Employee.JobSatisfaction)', 'SUM(Employee.MaritalStatus)', 'SUM(Employee.NumCompaniesWorked)', 'SUM(Employee.RelationshipSatisfaction)', 'SUM(Employee.StockOptionLevel)', 'SUM(Employee.WorkLifeBalance)', 'SUM(Employee.YearsInCurrentRole)', 'SUM(Employee.YearsSinceLastPromotion)', 'SUM(Employee.YearsWithCurrManager)']]

In [29]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import KFold,cross_val_score

from sklearn.metrics import confusion_matrix,classification_report, roc_auc_score,roc_curve,accuracy_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(new1,y,test_size=0.30,random_state=3,stratify=y)
lr=LogisticRegression()

#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
ytest_prob = lr.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))






cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "LR_FT_BACKWARD"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

LR_FT_BACKWARD =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

LR_FT_BACKWARD


Confusion: [[363   7]
 [ 50  21]]

Accuracy Train:  0.8794946550048591
Accuracy Test: 0.8707482993197279
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.98      0.93       370
           1       0.75      0.30      0.42        71

    accuracy                           0.87       441
   macro avg       0.81      0.64      0.68       441
weighted avg       0.86      0.87      0.85       441

AUC Test:  0.7979063570612865
True positives: 21
True negatives: 363
False positives (Type I error): 7
False negatives (Type II error): 50


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,LR_FT_BACKWARD,57,7,50,0.75,0.3,0.88,0.87,0.42,0.8


In [ ]:

X_train,X_test,y_train,y_test=train_test_split(new1,y,test_size=0.30,random_state=1,stratify=y)
lr=LogisticRegression()
lr_Bag=BaggingClassifier(base_estimator=lr,n_estimators=9,random_state=11)
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr_Bag.fit(X_train,y_train)
y_pred = lr_Bag.predict(X_test)
ytest_prob = lr_Bag.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr_Bag.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))



cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "Bag_LR_BACKWARD"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr_Bag.predict(X_train)),3)
accuracy_test = round(accuracy_score(y_test,y_pred),3)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

bag_LR_BACKWARD =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

bag_LR_BACKWARD




Confusion: [[357  13]
 [ 45  26]]

Accuracy Train:  0.8756073858114675
Accuracy Test: 0.8684807256235828
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.96      0.92       370
           1       0.67      0.37      0.47        71

    accuracy                           0.87       441
   macro avg       0.78      0.67      0.70       441
weighted avg       0.85      0.87      0.85       441

AUC Test:  0.8306813856109629
True positives: 26
True negatives: 357
False positives (Type I error): 13
False negatives (Type II error): 45


In [ ]:
fpr,tpr,thresholds=roc_curve(ytest,y_test_prob)
plt.plot(fpr,tpr,'g')
plt.plot(fpr,fpr,'r')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

In [222]:
LR_Bag_var=[]
LR_Bag_be=[]
for val in np.arange(1,100):
  LR=LogisticRegression()
  lr_Bag=BaggingClassifier(base_estimator=LR,n_estimators=val,random_state=0)
  kf =KFold(shuffle=True,n_splits=3,random_state=0)
  cv_results = cross_val_score(lr_Bag, new1, y,cv=kf, scoring='roc_auc')
  LR_Bag_var.append(np.std(cv_results,ddof=1))
  LR_Bag_be.append(np.mean(1-cv_results))
  


In [223]:
print('roc_auc :',1-np.min(LR_Bag_be))
print('Bias error :',np.min(LR_Bag_be))
print('Variance error :',np.min(LR_Bag_var))

roc_auc : 0.8193417077615326
Bias error : 0.18065829223846738
Variance error : 0.0023614720943689743


In [30]:
lr=LogisticRegression()
X_train,X_test,y_train,y_test=train_test_split(new1,y,test_size=0.30,random_state=1)
lr_Ada=AdaBoostClassifier(base_estimator=lr,n_estimators=7,random_state=10)

#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr_Ada.fit(X_train,y_train)
y_pred = lr_Ada.predict(X_test)
ytest_prob = lr_Ada.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr_Ada.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))




cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "Ada_LR_BACKWARD"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr_Ada.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

Ada_LR_BACKWARD =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

Ada_LR_BACKWARD


Confusion: [[363   1]
 [ 68   9]]

Accuracy Train:  0.8620019436345967
Accuracy Test: 0.8435374149659864
Classification Report:
               precision    recall  f1-score   support

           0       0.84      1.00      0.91       364
           1       0.90      0.12      0.21        77

    accuracy                           0.84       441
   macro avg       0.87      0.56      0.56       441
weighted avg       0.85      0.84      0.79       441

AUC Test:  0.8030540887683745
True positives: 9
True negatives: 363
False positives (Type I error): 1
False negatives (Type II error): 68


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,Ada_LR_BACKWARD,69,1,68,0.9,0.12,0.86,0.84,0.21,0.8


### RFE

In [156]:
# type your code here
#RFE
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score
xtrain,xtest,ytrain,ytest=train_test_split(feature_matrix,y,test_size=0.3,random_state=3)


In [157]:
for i in [2,3,4,5,6,7,8,9,10,11,12]:
    lr=LogisticRegression()
    rfe_mod=RFE(estimator=lr,n_features_to_select=i)
    rfe_feat_mod=rfe_mod.fit(xtrain,ytrain)
    rank=rfe_feat_mod.ranking_
    res=pd.DataFrame()
    res['feature']=xtrain.columns
    res['rank']=rank
    col=res[res['rank']==1]['feature']
    lr=LogisticRegression()
    lr.fit(xtrain[col],ytrain)
    ypred=lr.predict(xtest[col])
    print('feature subset : ',i)
    print('R2 : ',r2_score(ytest,ypred))

feature subset :  2
R2 :  -0.20491803278688536
feature subset :  3
R2 :  -0.10852459016393445
feature subset :  4
R2 :  -0.07639344262295089
feature subset :  5
R2 :  -0.04426229508196733
feature subset :  6
R2 :  -0.04426229508196733
feature subset :  7
R2 :  -0.04426229508196733
feature subset :  8
R2 :  -0.04426229508196733
feature subset :  9
R2 :  -0.04426229508196733
feature subset :  10
R2 :  0.05213114754098347
feature subset :  11
R2 :  -0.028196721311475548
feature subset :  12
R2 :  -0.028196721311475548


In [158]:
rfe_mod=RFE(estimator=lr,n_features_to_select=60)
rfe_feat_mod1=rfe_mod.fit(xtrain,ytrain)
rank1=rfe_feat_mod1.ranking_
res1=pd.DataFrame()
res1['feature']=xtrain.columns
res1['rank1']=rank1
col=res1[res1['rank1']==1]['feature']
col.values

array(['MonthlyIncome', 'OverTime', 'TotalWorkingYears', 'Age',
       'COUNT(Employee)', 'MAX(Employee.BusinessTravel)',
       'MAX(Employee.Department)',
       'MAX(Employee.EnvironmentSatisfaction)', 'MAX(Employee.Gender)',
       'MAX(Employee.JobInvolvement)', 'MAX(Employee.JobLevel)',
       'MAX(Employee.JobSatisfaction)', 'MAX(Employee.MaritalStatus)',
       'MAX(Employee.NumCompaniesWorked)',
       'MAX(Employee.PerformanceRating)',
       'MAX(Employee.RelationshipSatisfaction)',
       'MAX(Employee.StockOptionLevel)',
       'MAX(Employee.TrainingTimesLastYear)',
       'MAX(Employee.WorkLifeBalance)',
       'MAX(Employee.YearsInCurrentRole)', 'MEAN(Employee.Department)',
       'MEAN(Employee.EnvironmentSatisfaction)', 'MEAN(Employee.Gender)',
       'MEAN(Employee.JobInvolvement)', 'MEAN(Employee.JobLevel)',
       'MEAN(Employee.JobSatisfaction)', 'MEAN(Employee.MaritalStatus)',
       'MEAN(Employee.NumCompaniesWorked)',
       'MEAN(Employee.PerformanceRating)',
 

In [159]:
new2=feature_matrix[['MonthlyIncome', 'OverTime', 'TotalWorkingYears', 'Age',
       'COUNT(Employee)', 'MAX(Employee.BusinessTravel)',
       'MAX(Employee.Department)',
       'MAX(Employee.EnvironmentSatisfaction)', 'MAX(Employee.Gender)',
       'MAX(Employee.JobInvolvement)', 'MAX(Employee.JobLevel)',
       'MAX(Employee.JobSatisfaction)', 'MAX(Employee.MaritalStatus)',
       'MAX(Employee.NumCompaniesWorked)',
       'MAX(Employee.PerformanceRating)',
       'MAX(Employee.RelationshipSatisfaction)',
       'MAX(Employee.StockOptionLevel)',
       'MAX(Employee.TrainingTimesLastYear)',
       'MAX(Employee.WorkLifeBalance)',
       'MAX(Employee.YearsInCurrentRole)', 'MEAN(Employee.Department)',
       'MEAN(Employee.EnvironmentSatisfaction)', 'MEAN(Employee.Gender)',
       'MEAN(Employee.JobInvolvement)', 'MEAN(Employee.JobLevel)',
       'MEAN(Employee.JobSatisfaction)', 'MEAN(Employee.MaritalStatus)',
       'MEAN(Employee.NumCompaniesWorked)',
       'MEAN(Employee.PerformanceRating)',
       'MEAN(Employee.RelationshipSatisfaction)',
       'MEAN(Employee.StockOptionLevel)',
       'MEAN(Employee.TrainingTimesLastYear)',
       'MEAN(Employee.WorkLifeBalance)',
       'MEAN(Employee.YearsInCurrentRole)',
       'MEAN(Employee.YearsSinceLastPromotion)',
       'MIN(Employee.Department)',
       'MIN(Employee.EnvironmentSatisfaction)', 'MIN(Employee.Gender)',
       'MIN(Employee.JobInvolvement)', 'MIN(Employee.JobLevel)',
       'MIN(Employee.JobSatisfaction)', 'MIN(Employee.MaritalStatus)',
       'MIN(Employee.NumCompaniesWorked)',
       'MIN(Employee.PerformanceRating)',
       'MIN(Employee.RelationshipSatisfaction)',
       'MIN(Employee.StockOptionLevel)', 'MIN(Employee.WorkLifeBalance)',
       'MIN(Employee.YearsAtCompany)', 'MIN(Employee.YearsInCurrentRole)',
       'MIN(Employee.YearsWithCurrManager)', 'SUM(Employee.Department)',
       'SUM(Employee.EnvironmentSatisfaction)', 'SUM(Employee.Gender)',
       'SUM(Employee.JobInvolvement)', 'SUM(Employee.JobLevel)',
       'SUM(Employee.JobSatisfaction)', 'SUM(Employee.MaritalStatus)',
       'SUM(Employee.PerformanceRating)',
       'SUM(Employee.RelationshipSatisfaction)',
       'SUM(Employee.WorkLifeBalance)']]

In [160]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import KFold,cross_val_score

from sklearn.metrics import confusion_matrix,classification_report, roc_auc_score,roc_curve,accuracy_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(new2,y,test_size=0.30,random_state=3,stratify=y)
lr=LogisticRegression()

#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
ytest_prob = lr.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))






cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description ='LR_FT_RFE'
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

LR_FT_RFE =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

LR_FT_RFE


Confusion: [[364   6]
 [ 49  22]]

Accuracy Train:  0.8775510204081632
Accuracy Test: 0.8752834467120182
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.98      0.93       370
           1       0.79      0.31      0.44        71

    accuracy                           0.88       441
   macro avg       0.83      0.65      0.69       441
weighted avg       0.87      0.88      0.85       441

AUC Test:  0.7820708031975637
True positives: 22
True negatives: 364
False positives (Type I error): 6
False negatives (Type II error): 49


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,LR_FT_RFE,55,6,49,0.79,0.31,0.88,0.88,0.44,0.78


In [161]:

X_train,X_test,y_train,y_test=train_test_split(new2,y,test_size=0.30,random_state=3,stratify=y)
lr=LogisticRegression()
lr_Bag=BaggingClassifier(base_estimator=lr,n_estimators=9,random_state=0)
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr_Bag.fit(X_train,y_train)
y_pred = lr_Bag.predict(X_test)
ytest_prob = lr_Bag.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr_Bag.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))



cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "bag_LR_RFE"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr_Bag.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

bag_LR_RFE =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

bag_LR_RFE


Confusion: [[364   6]
 [ 51  20]]

Accuracy Train:  0.8756073858114675
Accuracy Test: 0.8707482993197279
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.98      0.93       370
           1       0.77      0.28      0.41        71

    accuracy                           0.87       441
   macro avg       0.82      0.63      0.67       441
weighted avg       0.86      0.87      0.84       441

AUC Test:  0.7796345641416064
True positives: 20
True negatives: 364
False positives (Type I error): 6
False negatives (Type II error): 51


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,bag_LR_RFE,57,6,51,0.77,0.28,0.88,0.87,0.41,0.78


In [162]:
lr=LogisticRegression()
lr_Ada=AdaBoostClassifier(base_estimator=lr,n_estimators=7,random_state=0)
X_train,X_test,y_train,y_test=train_test_split(new2,y,test_size=0.30,random_state=3)
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr_Ada.fit(X_train,y_train)
y_pred = lr_Ada.predict(X_test)
ytest_prob = lr_Ada.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr_Ada.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))




cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "Ada_LR_RFE"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr_Ada.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

Ada_LR_RFE =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

Ada_LR_RFE


Confusion: [[365   1]
 [ 62  13]]

Accuracy Train:  0.86977648202138
Accuracy Test: 0.8571428571428571
Classification Report:
               precision    recall  f1-score   support

           0       0.85      1.00      0.92       366
           1       0.93      0.17      0.29        75

    accuracy                           0.86       441
   macro avg       0.89      0.59      0.61       441
weighted avg       0.87      0.86      0.81       441

AUC Test:  0.7758105646630238
True positives: 13
True negatives: 365
False positives (Type I error): 1
False negatives (Type II error): 62


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,Ada_LR_RFE,63,1,62,0.93,0.17,0.87,0.86,0.29,0.78


In [169]:
new3=feature_matrix[['OverTime', 'TotalWorkingYears', 'HourlyRate', 'Age', 'DistanceFromHome', 'COUNT(Employee)', 'MAX(Employee.EnvironmentSatisfaction)', 'MAX(Employee.Gender)', 'MAX(Employee.JobInvolvement)', 'MAX(Employee.JobSatisfaction)', 'MAX(Employee.MaritalStatus)', 'MAX(Employee.NumCompaniesWorked)', 'MAX(Employee.RelationshipSatisfaction)', 'MAX(Employee.StockOptionLevel)', 'MAX(Employee.TrainingTimesLastYear)', 'MAX(Employee.YearsAtCompany)', 'MAX(Employee.YearsInCurrentRole)', 'MAX(Employee.YearsWithCurrManager)', 'MEAN(Employee.EnvironmentSatisfaction)', 'MEAN(Employee.Gender)', 'MEAN(Employee.JobInvolvement)', 'MEAN(Employee.JobSatisfaction)', 'MEAN(Employee.MaritalStatus)', 'MEAN(Employee.NumCompaniesWorked)', 'MEAN(Employee.StockOptionLevel)', 'MEAN(Employee.TrainingTimesLastYear)', 'MEAN(Employee.YearsAtCompany)', 'MEAN(Employee.YearsInCurrentRole)', 'MIN(Employee.EnvironmentSatisfaction)', 'MIN(Employee.Gender)', 'MIN(Employee.JobInvolvement)', 'MIN(Employee.JobSatisfaction)', 'MIN(Employee.MaritalStatus)', 'MIN(Employee.NumCompaniesWorked)', 'MIN(Employee.RelationshipSatisfaction)', 'MIN(Employee.StockOptionLevel)', 'MIN(Employee.YearsAtCompany)', 'MIN(Employee.YearsInCurrentRole)', 'MIN(Employee.YearsWithCurrManager)', 'SUM(Employee.Education)', 'SUM(Employee.EnvironmentSatisfaction)', 'SUM(Employee.Gender)', 'SUM(Employee.JobSatisfaction)', 'SUM(Employee.MaritalStatus)', 'SUM(Employee.NumCompaniesWorked)', 'SUM(Employee.RelationshipSatisfaction)', 'SUM(Employee.StockOptionLevel)', 'SUM(Employee.TrainingTimesLastYear)', 'SUM(Employee.YearsAtCompany)', 'SUM(Employee.YearsInCurrentRole)', 'SUM(Employee.YearsWithCurrManager)']]

In [170]:
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.model_selection import KFold,cross_val_score

from sklearn.metrics import confusion_matrix,classification_report, roc_auc_score,roc_curve,accuracy_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(new3,y,test_size=0.30,random_state=3,stratify=y)
lr=LogisticRegression()

#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
ytest_prob = lr.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))






cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description ='LR_FT_FS'
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

LR_FT_FS =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

LR_FT_FS


Confusion: [[362   8]
 [ 57  14]]

Accuracy Train:  0.8814382896015549
Accuracy Test: 0.8526077097505669
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.98      0.92       370
           1       0.64      0.20      0.30        71

    accuracy                           0.85       441
   macro avg       0.75      0.59      0.61       441
weighted avg       0.83      0.85      0.82       441

AUC Test:  0.7847735059002664
True positives: 14
True negatives: 362
False positives (Type I error): 8
False negatives (Type II error): 57


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,LR_FT_FS,65,8,57,0.64,0.2,0.88,0.85,0.3,0.78


In [171]:

X_train,X_test,y_train,y_test=train_test_split(new3,y,test_size=0.30,random_state=3,stratify=y)
lr=LogisticRegression()
lr_Bag=BaggingClassifier(base_estimator=lr,n_estimators=9,random_state=0)
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr_Bag.fit(X_train,y_train)
y_pred = lr_Bag.predict(X_test)
ytest_prob = lr_Bag.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr_Bag.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))



cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "bag_LR_FS"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr_Bag.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

bag_LR_FS =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

bag_LR_FS


Confusion: [[363   7]
 [ 56  15]]

Accuracy Train:  0.8794946550048591
Accuracy Test: 0.8571428571428571
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.98      0.92       370
           1       0.68      0.21      0.32        71

    accuracy                           0.86       441
   macro avg       0.77      0.60      0.62       441
weighted avg       0.84      0.86      0.82       441

AUC Test:  0.7819946707270652
True positives: 15
True negatives: 363
False positives (Type I error): 7
False negatives (Type II error): 56


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,bag_LR_FS,63,7,56,0.68,0.21,0.88,0.86,0.32,0.78


In [172]:
lr=LogisticRegression()
lr_Ada=AdaBoostClassifier(base_estimator=lr,n_estimators=7,random_state=0)
X_train,X_test,y_train,y_test=train_test_split(new3,y,test_size=0.30,random_state=3)
#from imblearn.over_sampling import SMOTE
#sm = SMOTE(random_state=0)
#X_train,y_train=sm.fit_sample(X_train,y_train)

lr_Ada.fit(X_train,y_train)
y_pred = lr_Ada.predict(X_test)
ytest_prob = lr_Ada.predict_proba(X_test)[:,1]
print("Confusion:",confusion_matrix(y_test,y_pred))
print()
print("Accuracy Train: ",accuracy_score(y_train,lr_Ada.predict(X_train)))
print("Accuracy Test:",accuracy_score(y_test,y_pred))
print("Classification Report:\n",classification_report(y_test,y_pred))
print("AUC Test: ",roc_auc_score(y_test,ytest_prob))




cm = pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted:0','Predicted:1'], index=['Actual:0','Actual:1'])

# calculating TP,TN,FP,FN
TN, FP, FN, TP = cm.iloc[0,0], cm.iloc[0,1], cm.iloc[1,0], cm.iloc[1,1]

# print values
print("True positives:", TP)
print("True negatives:", TN)
print("False positives (Type I error):", FP)
print("False negatives (Type II error):", FN)


from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve

# itereation results
description = "Ada_LR_FS"
misclassifications = FP + FN
type1 = FP
type2 = FN
precision = round(precision_score(y_test,y_pred),2)
recall = round(recall_score(y_test,y_pred),2)
accuracy_train = round(accuracy_score(y_train,lr_Ada.predict(X_train)),2)
accuracy_test = round(accuracy_score(y_test,y_pred),2)
f1 = round(f1_score(y_test,y_pred),2)
auc = round(roc_auc_score(y_test,ytest_prob),2)

Ada_LR_FS =pd.DataFrame(np.array([description,
                                     misclassifications,
                                     type1,
                                     type2,
                                     precision,
                                     recall,
                                     accuracy_train,
                                     accuracy_test,
                                     f1,
                                     auc]).reshape(1,-1), columns=['Description','Misclassifications','Type I errors','Type II errors','Precision','Recall','Accuracy_train','Accuracy_test','F1-score','ROC AUC'])

Ada_LR_FS


Confusion: [[365   1]
 [ 58  17]]

Accuracy Train:  0.8639455782312925
Accuracy Test: 0.8662131519274376
Classification Report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.93       366
           1       0.94      0.23      0.37        75

    accuracy                           0.87       441
   macro avg       0.90      0.61      0.65       441
weighted avg       0.88      0.87      0.83       441

AUC Test:  0.7642258652094718
True positives: 17
True negatives: 365
False positives (Type I error): 1
False negatives (Type II error): 58


,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,Ada_LR_FS,59,1,58,0.94,0.23,0.86,0.87,0.37,0.76


In [216]:
df_results=pd.DataFrame()
df_results=df_results.append(Ada_LR_FS, ignore_index=True)
df_results=df_results.append(bag_LR_FS, ignore_index=True)
df_results=df_results.append(LR_FT_FS, ignore_index=True)
df_results=df_results.append(Ada_LR_RFE, ignore_index=True)
df_results=df_results.append(bag_LR_RFE, ignore_index=True)
df_results=df_results.append(LR_FT_RFE, ignore_index=True)
df_results=df_results.append(Ada_LR_BACKWARD, ignore_index=True)
df_results=df_results.append(bag_LR_BACKWARD, ignore_index=True)
df_results=df_results.append(LR_FT_BACKWARD, ignore_index=True)
df_results=df_results.append(Ada_LR_SELECT, ignore_index=True)
df_results=df_results.append(bag_LR_SELECT, ignore_index=True)
df_results=df_results.append(LR_FT_SELECT, ignore_index=True)


df_results

,Description,Misclassifications,Type I errors,Type II errors,Precision,Recall,Accuracy_train,Accuracy_test,F1-score,ROC AUC
0,Ada_LR_FS,59,1,58,0.94,0.23,0.86,0.87,0.37,0.76
1,bag_LR_FS,63,7,56,0.68,0.21,0.88,0.86,0.32,0.78
2,LR_FT_FS,65,8,57,0.64,0.2,0.88,0.85,0.3,0.78
3,Ada_LR_RFE,63,1,62,0.93,0.17,0.87,0.86,0.29,0.78
4,bag_LR_RFE,57,6,51,0.77,0.28,0.88,0.87,0.41,0.78
5,LR_FT_RFE,55,6,49,0.79,0.31,0.88,0.88,0.44,0.78
6,Ada_LR_BACKWARD,61,2,59,0.89,0.21,0.86,0.86,0.34,0.77
7,Bag_LR_BACKWARD,58,13,45,0.67,0.37,0.88,0.87,0.47,0.83
8,LR_FT_BACKWARD,57,7,50,0.75,0.3,0.88,0.87,0.42,0.8
9,Ada_LR_SELECT,64,2,62,0.87,0.17,0.85,0.85,0.29,0.72


In [ ]:
from scipy.stats import f_oneway
tcol=[]
tpval=[]
ts=[]
for n in feature_matrix.select_dtypes(include='number').columns:
    tcol.append(n)
    g1=feature_matrix[n][feature_matrix['Attrition']==1]
    g2=feature_matrix[n][feature_matrix['Attrition']==0]
    
    s,p=f_oneway(g1,g2)
    tpval.append(round(p,4))
    if (p<=0.05):
        ts.append('Significant')
    else:
        ts.append('Insignificant')
print(pd.DataFrame({'Feature':tcol,'p-value':tpval,'Significance':ts}))

In [ ]:
df1['Attrition']